# Experiment with extracting shape info from confocal images of fish embryos

In [ ]:
from aicsimageio import AICSImage
import os 
import glob2 as glob

# define path to image
image_name = "2024_StayGold_10x_s018_T0"
read_path = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/morphSeq/data/yx1_samples/20230412/"

image_path = read_path + image_name + ".ome.tif"

# set path to segmentation
seg_path = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/morphSeq/data/20230412/seg_24hpf_classifier/"

# set save path
pcd_path = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/morphSeq/data/yx1_pcd/20230412/"

# make save directory
if not os.path.isdir(pcd_path):
    os.makedirs(pcd_path)

# get list of segmentation files
seg_list = sorted(glob.glob(seg_path + "*.tif"))

# set time points
time_ind = 0
labelObject = AICSImage(seg_list[time_ind])


In [ ]:
import plotly.graph_objects as go
from skimage.morphology import (erosion, dilation, opening, closing, white_tophat)
import plotly.express as px
from skimage import data
from skimage import filters
from skimage.segmentation import active_contour
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
from skimage.morphology import black_tophat, skeletonize, convex_hull_image 
from skimage.morphology import disk
import skimage
import scipy
import numpy as np

# get resolution
res_raw = labelObject.physical_pixel_sizes
res_array = np.asarray(res_raw)
res_array = np.insert(res_array, 0, 1)
pixel_size_z = 20#res_array[1]
pixel_size_x = 1#res_array[2]
pixel_size_y = 1#res_array[3]

# extract raw image
labelData = np.squeeze(labelObject.data)

# resize image such that voxels are isotropic
z_rs_factor = pixel_size_z/pixel_size_x
ds_factor = 4
pixel_size_new = pixel_size_x / ds_factor
labelData_rs = scipy.ndimage.zoom(labelData, [z_rs_factor/ds_factor, 1/ds_factor, 1/ds_factor])

labelDataBin = labelData_rs>0

keep_indices = np.where(labelDataBin ==1)

z_grid3, y_grid3, x_grid3 = np.meshgrid(range(0, labelDataBin.shape[0]), 
                                        range(0, labelDataBin.shape[1]), 
                                        range(0, labelDataBin.shape[2]),
                                       indexing="ij")

# # calculate position of max point for each
x_surf3 = x_grid3[keep_indices]
y_surf3 = y_grid3[keep_indices]
z_surf3 = z_grid3[keep_indices]


In [ ]:
# vizualise
max_z_project = np.max(labelDataBin, axis=0)
max_z_plane = np.divide(np.sum(np.multiply(labelDataBin,z_grid3), axis=0), np.sum(labelDataBin, axis=0))
max_z_plane = max_z_plane.astype(float)
max_z_plane[np.where(max_z_project==0)] = np.nan

plt.imshow(max_z_plane)

Attempt to infer a surface using marching cubes

**Can we use linear interpolation to fill label gaps?**

In [ ]:
from skimage.morphology import disk
from skimage.morphology import (erosion, dilation, opening, closing,  # noqa
                                white_tophat)

# first, generate 3D mask to identify interior pixels that we wish to interpolate
fish_mask = (np.max(labelDataBin, axis=0) == 1)*1

# use morphological closure operation to fill in shape
footprint = disk(13)
fp_small = disk(2)
fish_closed = closing(fish_mask > 0, footprint)
fish_clean = skimage.morphology.remove_small_objects(label(fish_closed), min_size=64)
fish_eroded = skimage.morphology.binary_erosion(fish_clean, fp_small)
# plt.imshow(fish_mask)
plt.imshow(fish_clean)

Use known z label locations to interpolate missing ones

In [ ]:
from scipy.interpolate import LinearNDInterpolator

y_grid2, x_grid2 = np.meshgrid(
                                range(0, max_z_plane.shape[0]), 
                                range(0, max_z_plane.shape[1]),
                                indexing="ij")

x_flat = x_grid2.flatten()
y_flat = y_grid2.flatten()

# generate diplacement vectors
max_z_plane_vals = max_z_plane.flatten()
mask_flat = (fish_clean.flatten() > 0)*1
mask_er = (fish_eroded.flatten() > 0)*1

ref_x = x_flat[np.where((~np.isnan(max_z_plane_vals)) & (mask_flat==1))]
ref_y = y_flat[np.where((~np.isnan(max_z_plane_vals)) & (mask_flat==1))]
ref_z = max_z_plane_vals[np.where((~np.isnan(max_z_plane_vals)) & (mask_flat==1))]

query_x = x_flat[np.where(mask_flat==1)]
query_y = y_flat[np.where(mask_flat==1)]
query_x_er = x_flat[np.where(mask_er==1)]
query_y_er = y_flat[np.where(mask_er==1)]

# interpolate
interp_z = LinearNDInterpolator(list(zip(ref_x, ref_y)), ref_z)
query_z = interp_z(query_x, query_y)
            

# disp_x = interp_x(source_xyz_norm[:, 0], source_xyz_norm[:, 1], source_xyz_norm[:, 2])
# disp_y = interp_y(source_xyz_norm[:, 0], source_xyz_norm[:, 1], source_xyz_norm[:, 2])
# disp_z = interp_z(source_xyz_norm[:, 0], source_xyz_norm[:, 1], source_xyz_norm[:, 2])

In [ ]:
surf_mask_array_raw = np.zeros(labelData_rs.shape)
surf_mask_array_raw[np.round(query_z).astype(int), query_y, query_x] = 1

surf_mask_array = surf_mask_array_raw.copy() 
# surf_mask_array[40, query_y_er, query_x_er] = 1

plt.imshow(np.argmax(surf_mask_array, axis=0))

Maybe alphashape?

In [ ]:
from alphashape import alphashape
import open3d as o3d
z_flat = query_z#max_z_plane.flatten()
plot_x = query_x#[np.where((~np.isnan(max_z_plane_vals) & (mask_flat>0)))]
plot_y = query_y#[np.where((~np.isnan(max_z_plane_vals) & (mask_flat>0)))]
plot_z = z_flat#[np.where((~np.isnan(max_z_plane_vals) & (mask_flat>0)))]

surf_flat = surf_mask_array.flatten()
keep_indices = np.where(surf_mask_array>0)
# x3 = x_grid3[keep_indices]
# y3 = y_grid3[keep_indices]
# z3 = z_grid3[keep_indices]
# generate smooth bottom
base_array = np.empty((len(plot_x), 3))
base_array[:, 0] = plot_x
base_array[:, 1] = plot_y
base_array[:, 2] = plot_z + 10

xyz_array3 = np.concatenate((plot_x[:, np.newaxis], plot_y[:, np.newaxis], plot_z[:, np.newaxis]), axis=1)
# xyz_array3 = np.concatenate((x_surf3[:, np.newaxis], y_surf3[:, np.newaxis], z_surf3[:, np.newaxis]), axis=1)
xyz_array3 = np.concatenate((xyz_array3, base_array), axis=0)
xyz_array3_norm = xyz_array3.copy()
xyz_array3_norm = xyz_array3_norm / np.max(xyz_array3_norm)

print(xyz_array3_norm.shape)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_array3_norm)
pcd_ds = pcd.voxel_down_sample(voxel_size=0.0001)

xyz_ds = np.asarray(pcd_ds.points)
print(xyz_ds.shape)
alpha_fish = alphashape(xyz_ds, alpha=65)
alpha_fish.show()

In [ ]:
import trimesh
# trimesh.repair.fill_holes(alpha_fish)
# print(test)
mesh_out = trimesh.smoothing.filter_taubin(alpha_fish, iterations=10)
# mesh_out = trimesh.smoothing.get_vertices_normals(mesh_out)
mesh_out.show()
print(mesh_out)

In [ ]:
test = trimesh.load_mesh('test_mesh.stl')
test.show()

In [ ]:
# smoothed_surface = nppas.smooth_surface_moving_least_squares_2d(e_surf2)
# smoothed_surface
simplified_surface = nppas.decimate_quadric(e_surf2, fraction=0.1)
simplified_surface

In [ ]:
smoothed_surface = nppas.smooth_surface_moving_least_squares_2d(simplified_surface, smoothing_factor=0.2)
smoothed_surface

Well...let's try a different method

In [ ]:
from skimage.filters import gaussian

surf_mask_array_smooth = np.argmax(surf_mask_array_raw.copy(), axis=0)
surf_mask_array_smooth[np.where(surf_mask_array_smooth==0)] = 84
surf_mask_array_smooth = gaussian(surf_mask_array_smooth, sigma=2, preserve_range=True)
plt.imshow(surf_mask_array_smooth)
new_z = surf_mask_array_smooth[query_y_er, query_x_er]

In [ ]:
xyz_array = np.concatenate((query_x_er[:, np.newaxis], query_y_er[:, np.newaxis], new_z[:, np.newaxis]), axis=1)

base_array = np.empty((len(query_x_er), 3))
base_array[:, 0] = query_x_er
base_array[:, 1] = query_y_er
base_array[:, 2] = new_z + 20

# xyz_array3 = np.concatenate((plot_x[:, np.newaxis], plot_y[:, np.newaxis], plot_z[:, np.newaxis]), axis=1)
# xyz_array3 = np.concatenate((x_surf3[:, np.newaxis], y_surf3[:, np.newaxis], z_surf3[:, np.newaxis]), axis=1)
xyz_array3 = np.concatenate((xyz_array, base_array), axis=0)
xyz_array3_norm = xyz_array3.copy()
xyz_array3_norm = xyz_array3_norm / np.max(xyz_array3_norm)

print(xyz_array3_norm.shape)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_array3_norm)
pcd_ds = pcd.voxel_down_sample(voxel_size=0.0001)
cl, ind = pcd_ds.remove_statistical_outlier(nb_neighbors=20,
                                                        std_ratio=2.0)

xyz_ds = np.asarray(pcd_ds.points)
print(xyz_ds.shape)
alpha_fish = alphashape(xyz_ds[ind], alpha=50)
alpha_fish.show()

In [ ]:
alpha_fish

In [ ]:
import pyvista as pv
import open3d as o3d

xyz_array = np.concatenate((query_x_er[:, np.newaxis], query_y_er[:, np.newaxis], new_z[:, np.newaxis]), axis=1)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_array)
pcd_ds = pcd.voxel_down_sample(voxel_size=1)

xyz_ds = np.asarray(pcd_ds.points)
pv_cloud = pv.PolyData(xyz_ds)
print(xyz_ds.shape)
print(xyz_array.shape)


alpha = 10
dl_3d_mesh = pv_cloud.delaunay_2d(alpha=alpha)
# # dl_3d_mesh = alphashape.alphashape(xyz_in, alpha=8)
# dl_3d_mesh.fill_holes(200, inplace=True)
smooth = dl_3d_mesh.smooth_taubin(n_iter=5000)
smooth.plot(jupyter_backend='ipygany', scalars=xyz_ds[:, 2])

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(int)

In [ ]:
(mnist.data.shape)

In [ ]:
nppas.fill_holes_in_surface(smoothed_surface, size_limit=300)

In [ ]:
simplified_surface2 = nppas.decimate_quadric(smoothed_surface, fraction=0.01)
simplified_surface2

**Let's see if we can fill gaps using linear interpolation**

In [ ]:
from skimage.morphology import disk
from skimage.morphology import (erosion, dilation, opening, closing,  # noqa
                                white_tophat)

# first, generate 3D mask to identify interior pixels that we wish to interpolate
fish_mask = (np.max(imDataBin, axis=0) == 1)*1

# use morphological closure operation to fill in shape
footprint = disk(15)
fish_closed = closing(fish_mask, footprint)

# plt.imshow(fish_mask)
plt.imshow(fish_closed)


In [ ]:
from scipy import interpolate

# identify "holes" for which not a single pixel nonzero exists 
query_indices = np.where((fish_closed==1) & (fish_mask!=1))[0]
ref_indices = np.where(fish_mask==1)[0]
print(len(candidate_indices))
print(len(ref_indices))

f = interpolate.interp2d(xyz_array2[:, 0], xyz_array2[:, 1], xyz_array2[:, 2], kind='linear')
x_query = x_grid2[query_indices]
y_query = y_grid2[query_indices]
z_query = f(x_query, y_query)

Try fitting to the surface points only

In [ ]:
surf_mask_array = np.zeros(imData_rs.shape)
sub_i = np.roll(xyz_array2, 1, axis=1)
surf_indices = np.ravel_multi_index((1,2,2), imData_rs.shape)
surf_mask_array[xyz_array2[:, 2], xyz_array2[:, 1], xyz_array2[:, 0]] = 1

e_surf = nppas.all_labels_to_surface(surf_mask_array)
e_surf

In [ ]:
e_surf.__dict__.keys()

**Attempt 2:** downsample, interpolate, and then infer mesh

In [ ]:
# import numpy as np
# import scipy
# from scipy import interpolate


# f = interpolate.interp2d(xyz_ds_in[:, 0], xyz_ds_in[:, 1], xyz_ds_in[:, 2], kind='linear')


In [ ]:
# npoints = 150
# xg = np.linspace(np.min(xyz_ds_in[:, 0]), np.max(xyz_ds_in[:, 0]), npoints)
# yg = np.linspace(np.min(xyz_ds_in[:, 1]), np.max(xyz_ds_in[:, 1]), npoints)

# x_grid, y_grid = np.meshgrid(xg, yg)

# znew = f(xg, yg)

# xyz_interp = np.concatenate((np.reshape(x_grid, (x_grid.size, 1)), 
#                              np.reshape(y_grid, (x_grid.size, 1)),
#                              np.reshape(znew, (x_grid.size, 1))),
#                              axis=1)

# xyz_interp_filt = xyz_interp[np.where(xyz_interp[:, 2]<=350)]
# xyz_interp_filt = xyz_interp_filt[xyz_interp_filt[:, 2]>0]
# pv_cloud_interp = pv.PolyData(xyz_interp_filt)
# pv_cloud_interp.plot(jupyter_backend='ipygany')